In [15]:
import json
import os
import re

In [16]:
# step 1 is to get instagram scraper and get metadata in the json format
#  instagram-scraper handlename/username -t none --media-metadata --maximum 30 
#instagram-scraper liltrails -t none --media-metadata --maximum 30

In [17]:
# Opening JSON file 
#path = os.getcwd();
#print(path)

path = "/Users/alex/Documents/live-event-chat/"
f = open(path + "/samarpanaforartsandwellbeing/samarpanaforartsandwellbeing.json"); 

# returns JSON object as  
# a dictionary 
data = json.load(f) 

In [18]:
# parser for natural language - feature extractor
# feature 1 - time to indicate that event is happening live 
# time: 5PM, 5pm, 5 PM, 5 pm, 5:30pm, 5:30PM, 5:30 PM, 5:30 pm, 5:30  PM, 5:30  pm,


class event:
  def __init__(liveEvent, invite, live, nonspecificday, day, month, time, synopsis, eventName, link, category):
    liveEvent.invite = invite
    liveEvent.live = live
    liveEvent.synopsis = synopsis
    liveEvent.month = month
    liveEvent.time =  time
    liveEvent.day = day
    liveEvent.event_name = eventName
    liveEvent.nonspecificday = nonspecificday
    liveEvent.link = link
    liveEvent.category = category 

parsedObj = []
def parseData ( text, eventName, link ):
   time = re.findall("((\d{1,4}|\d{1,4}\S\d{1,4})\s*(pm|PM|p\.m\.|P\.M\.|am|AM|a\.m\.|A\.M\.))(\s{1,2}\S{1,3}\s{1,2})*((\d{1,4}|\d{1,4}\S\d{1,4})\s*(pm|PM|p\.m\.|P\.M\.|am|AM|a\.m\.|A\.M\.))*",text)
   live = re.findall("live|online",text)
   invite = re.findall("[jJ]oin[a-z]*|[Ss]ee you|[dD]on't miss|[dD]o not miss",text)
   day = re.findall("[Mm]onday|[Tt]uesday|[wW]ednesday|[Tt]hursday|[Ff]riday|[Ss]aturday|[Ss]unday",text)
   nonspecificday = re.findall("[t|T]oday|[t|T]omorrow|[mM]orning|[aA]fternoon|[eE]vening",text)
   month =  re.findall("(\d+)*\s*(th|nd|rd|st)*\s*\w*\s*([jJ]anuary|[fF]ebruary|[mM]arch|[Aa]pril|[M]ay|[jJ]une|[jJ]uly|[aA]ugust|[sS]eptember|[oO]ctober|[nN]ovember|[dD]ecember)\s*(\d+)*\s*(th|nd|rd|st)*",text)
   category = re.findall("([Aa]rt[s|S]*)|([wW]ell[ness|being]\w*|[wW]orkout\w*|heath\w*)|([c|C]hild\w*|[kK]id\w*)|([wW]orkshop\w*|[sS]ession\w*|[cC]onversation\w*)", text)
   return event( invite, live, nonspecificday, day, month, time, text, eventName, link, category);

# parsedObj.append(event( invite, live, nonspecificday, day, month, time, text, eventName, link, category));
#print( [invite, live, nonspecificday, day, month, time]) 
#time = re.findall("(\d{0,23}\s+pm|PM)|(\d{0,23}pm|PM)|(\d{0,23}:\d{0,60}pm|PM)|(\d{0,23}:\d{0,60}\s+pm|PM)|(\d{0,23}\S\d{0,60}pm|PM)|(\d{0,23}\S\d{0,60}pm|PM)", text)
       

In [19]:
#for obj in parsedObj: 
#    print(json.dumps(obj.__dict__))
# obj.invite, obj.live, obj.nonspecificday, obj.day, obj.month, obj.time, obj.event_name, obj.link)

In [20]:
def catResponse( i ): 
    switcher={
                0:'Art',
                1:'Wellbeing',
                2:'Children',
                3:'Workshops',
             }
    return switcher.get(i,"")

def findCategory(catlist):
    catArr = [-1,-1,-1,-1]; #no categories
    # four basic categories: art, wellbeing, children, workshops
    for item in catlist:
        if item[0] != '':
            catArr[0] = 0
        elif item[1] != '':
            catArr[1] = 1
        elif item[2] != '':
            catArr[2] = 2
        elif item[3] != '':
            catArr[3] = 3
    return catArr

catList = findCategory([["", "", "kiddos", ""], ["art", "", "", ""], ["art", "", "", ""], ["art", "", "", ""], ["art", "", "", ""]])
print(catList)
finalCategories = list(filter(lambda x: (x!= '') , list(map(catResponse,catList))));
print(finalCategories)

[0, -1, 2, -1]
['Art', 'Children']


In [21]:
class dbEvent:
   def __init__(liveEvent, parsedObj):
    liveEvent.synopsis = parsedObj.synopsis
    liveEvent.date = monthFormat(parsedObj.month) #from month object. Other idea: use posting date
    if checkArr(parsedObj.time) == None:
            liveEvent.start_time = "unknown"
    else:
            liveEvent.start_time = checkTime(parsedObj.time[0][1],parsedObj.time[0][2])
    if checkArr(parsedObj.time) == None:
            liveEvent.end_time =  "unknown"
    else:
            liveEvent.end_time = checkTime(parsedObj.time[0][4],parsedObj.time[0][5])
    liveEvent.day = checkArr(parsedObj.day)
    liveEvent.event_name = parsedObj.event_name
    liveEvent.nonspecificday = checkArr(parsedObj.nonspecificday)
    liveEvent.link = parsedObj.link
    liveEvent.category = parsedObj.category 

def checkArr (Arr):
    if len(Arr) == 0: 
        return None
    else: 
        return Arr;

def findMonth( i ): 
    switcher={
                "January": "01",
                "january": "01",
                "February": "02",
                "March": "03",
                "April": "04",
                "May": "05",
                "June": "06",
                "July": "07",
                "August": "08",
                "September": "09",
                "October": "10",
                "November": "11",
                "December": "12"
             }
    return switcher.get(i,"")
    
def monthFormat (Arr):
    if checkArr(Arr) == None:
        return "unknown"
    elif Arr[0][0] == "" and Arr[0][3] == "":
        return "noDate"
    else:
        if Arr[0][0]!='' and (Arr[0][0]!="th" or Arr[0][0]!="nd" or Arr[0][0]!="st" or Arr[0][0]!="rd"):
            if int(Arr[0][0]) > 0 and int(Arr[0][0]) <= 31:     
                return "{}/{}/2020".format(Arr[0][0], findMonth(Arr[0][2]))    
            else:
                return "noDate"
        elif Arr[0][3]!='' and (Arr[0][3]!="th" or Arr[0][3]!="nd" or Arr[0][3]!="st" or Arr[0][3]!="rd") :
            if int(Arr[0][3]) > 0 and int(Arr[0][3]) <= 31:     
                return "{}/{}/2020".format(Arr[0][3], findMonth(Arr[0][2]))    
            else:
                return "noDate"
        else:
            return "unknown"

# 8:30, 8, Am, or PM
def checkTime (time, ampm ):
    if len(time) == 0 or time == " ":
        return "unknown"
    elif len(time.split(":")) > 1:
        hours = time.split(":")[0]   
        mins = time.split(":")[1]
    else:
        hours = time[0]
        mins = "00"
    if int(hours) < 12 and len(re.findall("AM|am|A\.M\.|a\.m\.", ampm)) > 0:
        return "{}:{}".format(hours, mins)
    elif int(hours) < 12 and len(re.findall("PM|pm|P\.M\.|p\.m\.", ampm)) > 0:
        return "{}:{}".format((int(hours)+12)%24, mins)
    else:
        return "{}:{}".format(hours, mins)
    

In [22]:
path = "/Users/alex/Documents/live-event-chat/"
eventHandles = ["samarpanaforartsandwellbeing", "aalaap_concepts", "karaditales", "madrasphotoblogs"]
dbObjects = []

for item in eventHandles:
    f = open(path + "/" + item + "/" +  item + ".json"); 
    # returns JSON object as  
    # a dictionary 
    data = json.load(f) 

    dataText = []

    for dataEl in data['GraphImages']: 
        dataText.append(dataEl["edge_media_to_caption"]["edges"][0]["node"]["text"]) 

    eventName= item
    link = "http://www.instagram.com/{}".format(item)
    
    for text in dataText:
           parsedObj = parseData(text, eventName, link)
           dbObj = dbEvent(parsedObj)  
           dbObjects.append(dbObj);
           
    # Closing file 
    f.close() 

# arrays of dictionaries
json_string = json.dumps([ob.__dict__ for ob in dbObjects])

with open("database.json", "w") as outfile: 
        json.dump(json_string,outfile,indent=3,sort_keys=True);
print(json_string)

#with open("database.json", "w") as outfile: 
#    for iter in range(0, len(dbObjects)):
#         if iter != len(dbObjects)-1:
#             json.dump(dbObjects[iter].__dict__, outfile, indent=2, sort_keys=True,separators= (',', ': '))
#             outfile.write(',')
#         else: 
#             json.dump(dbObjects[iter].__dict__, outfile, indent=2, sort_keys=True,separators= (',', ': '))



[{"synopsis": "We are happy to introduce Sreelakshmy Govardhanan as the next dancer in the Art Heals series. She will be joining us live tomorrow (Friday, April 17th) at 6 p.m. IST.\n\nSreelakshmy\u00a0Govardhanan is a Kuchipudi artiste from\u00a0India and a disciple of Guru Sri Pasumarthy\u00a0Rattaiaha\u00a0Sarma.\nShe is widely known for her grace, exquisite footwork and\u00a0abhinaya, which comes alive during her dance performances.\n\nSreelakshmy has been the recipient of many coveted titles and honours such as \u2018Kalashree\u2019 from Kerala\u00a0Sangeetha Nataka \u00a0akademi-\u00a0state award\u00a0, Adithya Vikram Birla Kalakiran Puraskar, Silver conch from Keli Mumbai, Kalasagar Award, Best Dancer from Music academy Chennai, Binfeild\u00a0Endowment from\u00a0Narada\u00a0Gana\u00a0Sabha\u00a0Chennai, Bharatham\u00a0Yuva\u00a0Kalakar, Kala\u00a0Ratna, Singar\u00a0Mani,\u00a0Natya\u00a0Ratna,\u00a0Nalanada\u00a0Nrutya\u00a0Nipuna to\u00a0name a few.\n\nSreelakshmy is a graded a